In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
%%time
# create dict lemma -> root
import codecs
from bs4 import BeautifulSoup

awn_file = '/home/jvdzwaan/Downloads/arb2-lmf.xml'
with codecs.open(awn_file) as f:
    soup = BeautifulSoup(f, 'xml')
    
entries = soup.find_all('LexicalEntry')
print(len(entries))

In [ ]:
%%time
def is_root(element):
    #print element
    #print element.name
    if element.name == 'WordForm' and element['formType'] == 'root':
        return True
    return False

awn = {}

for entry in entries:
    lemma = entry.Lemma['writtenForm']
    root = entry.find(is_root)
    if root:
        root = root['writtenForm']
        awn[lemma] = root
print(len(awn))

In [ ]:
%%time
# load analysed words
analysed_words = pd.read_csv('/home/jvdzwaan/data/tmp/adh/merged/analysed_words.csv', encoding='utf-8')

In [ ]:
types = analysed_words['word'].unique()
print(len(types))

In [ ]:
%%time
types_in_awn = []
for t in types:
    if t in awn:
        types_in_awn.append(t)
print(len(types_in_awn))

In [ ]:
%%time
# select word analyses for words for which we can lookup the root in awn
awn_analyses = analysed_words.loc[analysed_words['word'].isin(types_in_awn)].copy()

In [ ]:
awn_analyses.head()

In [ ]:
%%time
analyses_per_word = awn_analyses.groupby(['file_name', 'position']).count()

In [ ]:
analyses_per_word.head()

In [ ]:
analysed_words['file_name'].nunique()

In [ ]:
# the total number of analysed words in the corpus
num_words_corpus = analysed_words.groupby(['file_name', 'position']).count().shape[0]
print(num_words_corpus)

In [ ]:
# the total number of analysed words in the corpus that can be looked up in awn
num_words_in_awn = analyses_per_word.shape[0]
print(num_words_in_awn)

In [ ]:
(num_words_in_awn+0.0)/(num_words_corpus+0.0)*100.0

In [ ]:
analyses_per_word['root'].plot.hist(figsize=(15,8), bins=range(analyses_per_word['root'].max()))

In [ ]:
# Hoeveel woorden hebben 1 analyse? Van alle woorden in het corpus, ook de niet geanalyseerde
single_analysis = np.sum(analyses_per_word['root'] == 1)
print('abs {}'.format(single_analysis))
print('percentage {}'.format(single_analysis/16899400.0*100.0))

In [ ]:
# 
print('abs {}'.format(single_analysis))
print('percentage {}'.format(single_analysis/(num_words_corpus+0.0)*100.0))

In [ ]:
(16899400-num_words_corpus+0.0)/16899400.0*100

In [ ]:
awn_analyses.head(25)

In [ ]:
%%time
awn_analyses['root_correct'] = awn_analyses.apply(lambda row: row['root']==awn[row['word']], axis=1)

In [ ]:
awn_analyses.head(25)

In [ ]:
np.sum(awn_analyses['root_correct'])

In [ ]:
awn_analyses.shape

In [ ]:
found_correct = awn_analyses.groupby(['file_name', 'position']).sum()

In [ ]:
found_correct.head()

In [ ]:
print found_correct.shape
np.sum(found_correct['root_correct'])

In [ ]:
# percentage woorden waarvan de juiste root tussen de voorgestelde roots zit
(np.sum(found_correct['root_correct'])+0.0)/found_correct.shape[0]*100

In [ ]:
single_analysis = found_correct[found_correct['root_length'] == 3]

In [ ]:
# percentage woorden met 1 analyse die ook de juiste is
(np.sum(single_analysis['root_correct'])+0.0)/single_analysis.shape[0]*100

In [ ]:
from nltk.corpus import wordnet as wn

print sorted(wn.langs())

In [ ]:
lemmas = wn.lemmas(u'باب', lang='arb') # door

In [ ]:
for lemma in lemmas:
    print lemma.synset()

In [ ]:
print lemmas

In [ ]:
[l.name() for l in wn.synset('heading.n.01').lemmas()]

In [ ]:
dog_lemma = wn.lemma(b'dog.n.01.c\xc3\xa3o'.decode('utf-8'), lang='por')
print dog_lemma.name()

In [ ]:
all_lemmas = wn.all_lemma_names(pos='n', lang='arb')

In [ ]:
print all_lemmas[23]

In [ ]:
with codecs.open('/home/jvdzwaan/Downloads/arb2-lmf.xml') as f:
    soup = BeautifulSoup(f, 'xml')

In [ ]:
print soup.find('LexicalEntry')